# Auto-generation Utilities

> Auto-generate folder_name.ipynb notebooks for nbdev project organization

In [ ]:
#| default_exp generators

In [ ]:
#| export
from __future__ import annotations
from pathlib import Path
from typing import List, Dict, Any, Optional
from datetime import datetime
from nbdev.config import get_config
from execnb.nbio import *
from fastcore.all import *
from cjm_nbdev_overview.core import *
from cjm_nbdev_overview.tree import *

## Notebook Generation

In [ ]:
#| export
def create_folder_notebook(folder_path: Path,           # Path to folder
                          title: str,                   # Notebook title
                          description: str              # Folder description
                          ) -> List[NbCell]:            # List of notebook cells
    "Create cells for a folder notebook with proper nbdev structure"
    cells = []
    
    # Title cell with description
    title_cell = mk_cell(
        f"# {title}\n\n> {description}",
        cell_type='markdown'
    )
    cells.append(title_cell)
    
    # Default export cell
    export_cell = mk_cell(
        f"#| default_exp {folder_path.name}.__init__",
        cell_type='code'
    )
    cells.append(export_cell)
    
    # Import cell
    import_cell = mk_cell(
        "#| export\n# This module serves as the package root for this folder",
        cell_type='code'
    )
    cells.append(import_cell)
    
    # Documentation cell
    doc_cell = mk_cell(
        f"## Overview\n\nThis folder contains notebooks related to {description.lower()}.",
        cell_type='markdown'
    )
    cells.append(doc_cell)
    
    # List notebooks in folder
    notebooks = get_notebook_files(folder_path, recursive=False)
    if notebooks:
        nb_list = []
        nb_list.append("## Notebooks in this folder:\n")
        for nb in notebooks:
            if nb.stem != folder_path.name:  # Skip the folder notebook itself
                nb_info = extract_notebook_info(nb)
                if nb_info.description:
                    nb_list.append(f"- `{nb.name}`: {nb_info.description}")
                else:
                    nb_list.append(f"- `{nb.name}`")
        
        if len(nb_list) > 1:  # Only add if there are other notebooks
            notebooks_cell = mk_cell(
                '\n'.join(nb_list),
                cell_type='markdown'
            )
            cells.append(notebooks_cell)
    
    # Export cell at the end
    export_final_cell = mk_cell(
        "#| hide\nimport nbdev; nbdev.nbdev_export()",
        cell_type='code'
    )
    cells.append(export_final_cell)
    
    return cells

## Folder Notebook Generation

In [ ]:
#| export
def generate_folder_notebook(folder_path: Path,         # Path to folder
                           title: Optional[str] = None, # Custom title
                           description: Optional[str] = None, # Custom description
                           overwrite: bool = False      # Overwrite existing
                           ) -> Path:                   # Path to created notebook
    "Generate a folder_name.ipynb notebook for a folder"
    # Ensure it's an absolute path
    if not folder_path.is_absolute():
        cfg = get_config()
        folder_path = cfg.nbs_path / folder_path
    
    # Check if folder exists
    if not folder_path.exists() or not folder_path.is_dir():
        raise ValueError(f"Folder {folder_path} does not exist")
    
    # Determine notebook path
    nb_path = folder_path / f"{folder_path.name}.ipynb"
    
    # Check if already exists
    if nb_path.exists() and not overwrite:
        raise FileExistsError(f"Notebook {nb_path} already exists. Use overwrite=True to replace.")
    
    # Default title and description
    if title is None:
        # Convert folder name to title case
        title = folder_path.name.replace('_', ' ').replace('-', ' ').title()
    
    if description is None:
        description = f"Notebooks and utilities for {title.lower()}"
    
    # Create notebook cells
    cells = create_folder_notebook(folder_path, title, description)
    
    # Create and save notebook
    nb = new_nb(cells)
    write_nb(nb, nb_path)
    
    print(f"Created notebook: {nb_path}")
    return nb_path

In [ ]:
#| export
def generate_all_folder_notebooks(base_path: Path = None, # Base path (defaults to nbs)
                                 recursive: bool = True,  # Include nested folders
                                 overwrite: bool = False, # Overwrite existing
                                 dry_run: bool = False    # Just show what would be created
                                 ) -> List[Path]:         # Created notebook paths
    "Generate folder notebooks for all folders that don't have them"
    if base_path is None:
        cfg = get_config()
        base_path = cfg.nbs_path
    
    created = []
    subdirs = get_subdirectories(base_path, recursive=recursive)
    
    for folder in subdirs:
        nb_path = folder / f"{folder.name}.ipynb"
        
        if not nb_path.exists() or overwrite:
            if dry_run:
                print(f"Would create: {nb_path}")
            else:
                try:
                    path = generate_folder_notebook(folder, overwrite=overwrite)
                    created.append(path)
                except Exception as e:
                    print(f"Error creating {nb_path}: {e}")
        else:
            if dry_run:
                print(f"Already exists: {nb_path}")
    
    if not dry_run:
        print(f"\nCreated {len(created)} folder notebooks")
    
    return created

## Interactive Folder Notebook Generation

In [ ]:
#| export
def interactive_folder_notebook_generator(base_path: Path = None  # Base path
                                        ) -> List[Path]:          # Created notebooks
    "Interactively generate folder notebooks with custom titles and descriptions"
    if base_path is None:
        cfg = get_config()
        base_path = cfg.nbs_path
    
    created = []
    subdirs = get_subdirectories(base_path, recursive=True)
    
    # Filter folders without notebooks
    folders_without_nb = []
    for folder in subdirs:
        nb_path = folder / f"{folder.name}.ipynb"
        if not nb_path.exists():
            folders_without_nb.append(folder)
    
    if not folders_without_nb:
        print("All folders already have their folder notebooks!")
        return created
    
    print(f"Found {len(folders_without_nb)} folders without folder notebooks:\n")
    
    for i, folder in enumerate(folders_without_nb, 1):
        rel_path = folder.relative_to(base_path)
        print(f"{i}. {rel_path}/")
    
    print("\nEnter folder numbers to create notebooks for (comma-separated, or 'all'):")
    choice = input("> ").strip()
    
    if choice.lower() == 'all':
        selected_folders = folders_without_nb
    else:
        try:
            indices = [int(x.strip()) - 1 for x in choice.split(',')]
            selected_folders = [folders_without_nb[i] for i in indices if 0 <= i < len(folders_without_nb)]
        except (ValueError, IndexError):
            print("Invalid selection")
            return created
    
    # Generate notebooks for selected folders
    for folder in selected_folders:
        print(f"\n--- Creating notebook for {folder.name} ---")
        
        # Ask for custom title
        default_title = folder.name.replace('_', ' ').replace('-', ' ').title()
        title = input(f"Title [{default_title}]: ").strip() or default_title
        
        # Ask for description
        default_desc = f"Notebooks and utilities for {title.lower()}"
        description = input(f"Description [{default_desc}]: ").strip() or default_desc
        
        try:
            path = generate_folder_notebook(folder, title=title, description=description)
            created.append(path)
        except Exception as e:
            print(f"Error: {e}")
    
    print(f"\nCreated {len(created)} folder notebooks")
    return created

## Testing

Let's test the generator functionality:

In [ ]:
#|eval: false
# First, let's check if there are any folders in our project
subdirs = get_subdirectories()
print(f"Found {len(subdirs)} subdirectories in nbs/")
for d in subdirs:
    nb_path = d / f"{d.name}.ipynb"
    exists = "✓" if nb_path.exists() else "✗"
    print(f"{exists} {d.name}/")

Found 0 subdirectories in nbs/


In [ ]:
#|eval: false
# Create a test folder to demonstrate the functionality
test_folder = Path("test_folder")
if not test_folder.exists():
    test_folder.mkdir()
    print(f"Created test folder: {test_folder}")
    
    # Create a sample notebook in the folder
    sample_nb = test_folder / "sample.ipynb"
    cells = [
        mk_cell("# Sample Notebook\n\n> A sample notebook for testing", cell_type='markdown'),
        mk_cell("print('Hello from sample notebook!')", cell_type='code')
    ]
    nb = new_nb(cells)
    write_nb(nb, sample_nb)
    print(f"Created sample notebook: {sample_nb}")
else:
    print(f"Test folder already exists: {test_folder}")

Created test folder: test_folder
Created sample notebook: test_folder/sample.ipynb


In [ ]:
#|eval: false
# Generate a folder notebook for the test folder
try:
    nb_path = generate_folder_notebook(
        test_folder, 
        title="Test Folder", 
        description="Testing folder notebook generation"
    )
    print(f"\nGenerated notebook at: {nb_path}")
    
    # Read and display the generated notebook content
    nb = read_nb(nb_path)
    print("\nGenerated notebook contents:")
    print("-" * 50)
    for i, cell in enumerate(nb.cells):
        print(f"\nCell {i+1} ({cell.cell_type}):")
        print(cell.source)
except Exception as e:
    print(f"Error: {e}")

Created notebook: /mnt/SN850X_8TB_EXT4/Projects/GitHub/cj-mills/cjm-nbdev-overview/nbs/test_folder/test_folder.ipynb

Generated notebook at: /mnt/SN850X_8TB_EXT4/Projects/GitHub/cj-mills/cjm-nbdev-overview/nbs/test_folder/test_folder.ipynb

Generated notebook contents:
--------------------------------------------------

Cell 1 (markdown):
# Test Folder

> Testing folder notebook generation

Cell 2 (code):
#| default_exp test_folder.__init__

Cell 3 (code):
#| export
# This module serves as the package root for this folder

Cell 4 (markdown):
## Overview

This folder contains notebooks related to testing folder notebook generation.

Cell 5 (markdown):
## Notebooks in this folder:

- `sample.ipynb`: A sample notebook for testing

Cell 6 (code):
#| hide
import nbdev; nbdev.nbdev_export()


In [ ]:
#|eval: false
# Clean up test folder
import shutil
if test_folder.exists():
    shutil.rmtree(test_folder)
    print(f"Cleaned up test folder: {test_folder}")

Cleaned up test folder: test_folder


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()